In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def load_csv(filename):
    csv_path = os.path.join(labels_path, filename)
    return pd.read_csv(csv_path)

In [ ]:
def extract_patches(roi, height, width, length):
    #patches = np.zeros((length, patch_size, 3), dtype = np.float32)
    patches = np.zeros((length, patch_size, patch_size, 3), dtype = np.float32)
    patch_iter = 0
    for i in range(0, height - patch_size + 1):
        for j in range(0, width - patch_size + 1):
            #patches[patch_iter : patch_iter + patch_size, :, :] = roi[i : i + patch_size, j : j + patch_size, :]
            patches[patch_iter, :, :, :] = roi[i : i + patch_size, j : j + patch_size, :]
            #patch_iter += patch_size
            patch_iter += 1
    return patches

In [ ]:
def process_rois(filename, folder, count):
    roi_data = load_csv(filename)
    originals_folder = os.path.join(originals_path, folder)
    labels_folder = os.path.join(labels_path, folder)
    
#     np.random.seed(42)
#     roi_data = roi_data.iloc[np.random.permutation(len(roi_data))]

    N_patches = sum((roi_data['height'] - patch_size + 1) 
                    * (roi_data['width'] - patch_size + 1))
    rois = np.zeros((N_patches, patch_size, patch_size, 3), dtype = np.float32)
    skeletons = np.zeros((N_patches, 1), dtype = np.float32)

    roi_iter = 0
    skelet_iter = 0
    for index, row in roi_data.iterrows():
        img = plt.imread(originals_folder + row['label'] + '.tif')
        height = row['height']
        width = row['width']
        patches_per_roi = (height - patch_size + 1) * (width - patch_size + 1)
        #length = patch_size * patches_per_roi
        length = patches_per_roi
#         rois[roi_iter : roi_iter + length, :, :] = extract_patches(img[row['ycoord'] :
#                                                                        row['ycoord'] + height, 
#                                                                        row['xcoord'] : 
#                                                                        row['xcoord'] + width, :],
#                                                                    height, width, length)
        rois[roi_iter : roi_iter + length, :, :, :] = extract_patches(img[row['ycoord'] :
                                                                       row['ycoord'] + height, 
                                                                       row['xcoord'] : 
                                                                       row['xcoord'] + width, :],
                                                                   height, width, length)
        roi_iter += length
        
        img = plt.imread(labels_folder + row['label'] + '_' + str(row['sample']) + '_' + 'skeletone.jpg')
        img_grey = np.dot(img[...,:3], [0.299, 0.587, 0.114])
        skeletons[skelet_iter : skelet_iter + patches_per_roi] = img_grey[d : height - d, 
                                                                          d : width - d].reshape(-1, 1)

        skelet_iter += patches_per_roi
        
    labels[count : count + N_patches] = skeletons
    #originals[count * patch_size : count * patch_size + N_patches * patch_size, :, :] = rois
    originals[count : count + N_patches, :, :, :] = rois
    count += N_patches
    return originals, labels, count



## Load the data, extract originals and labels

In [ ]:
import time
start_time = time.time()

originals_path = '/Users/vladarozova/Dropbox/PhD project/angiogenesis/samples for ridge detection'
labels_path = '/Users/vladarozova/Dropbox/PhD project/angiogenesis/Anna\'s results'
output = '/Users/vladarozova/Dropbox/PhD project/angiogenesis/datasets'

patch_size = 23
d = int((patch_size - 1)/2)

#originals = np.empty((0, patch_size, 3), float32)
#originals = np.zeros((21000000, patch_size, 3), dtype = np.float32)
originals = np.zeros((885000, patch_size, patch_size, 3), dtype = np.float32)
#labels = np.empty((0, 1), float32)
labels = np.zeros((885000, 1), dtype = np.float32)

originals, labels, count = process_rois('CS8_ROIs.csv', 'cellsuspensionED8/', 0)
print('count = ', count)

print("--- %s seconds ---" % (time.time() - start_time))



In [ ]:
labels = labels[:count, :]
print('labels shape: ', labels.shape)
#originals = originals[:count * patch_size, :, :]
originals = originals[: count, :, :, :]
print('originals shape: ', originals.shape)

In [ ]:
i = 0
plt.figure(figsize=(15, 10))
plt.subplot(141)
plt.imshow(originals[i, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(142)
plt.imshow(originals[i+1, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(143)
plt.imshow(originals[i+2, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(144)
plt.imshow(originals[i+3, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")

plt.figure(figsize=(15, 10))
plt.subplot(141)
plt.imshow(originals[i+4, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(142)
plt.imshow(originals[i+5, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(143)
plt.imshow(originals[i+6, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(144)
plt.imshow(originals[i+7, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.show()
print(originals[i:i+8, 11, 11, 1])
labels[i:i+8]

In [ ]:
i = 2000
plt.figure(figsize=(15, 10))
plt.subplot(141)
plt.imshow(originals[i, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(142)
plt.imshow(originals[i+1, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(143)
plt.imshow(originals[i+2, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(144)
plt.imshow(originals[i+3, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")

plt.figure(figsize=(15, 10))
plt.subplot(141)
plt.imshow(originals[i+4, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(142)
plt.imshow(originals[i+5, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(143)
plt.imshow(originals[i+6, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.subplot(144)
plt.imshow(originals[i+7, :, :, 1], cmap = plt.cm.gray, interpolation = "nearest")
plt.show()
print(originals[i:i+8, 11, 11, 1])
labels[i:i+8]

## Evaluate the data

In [ ]:
plt.hist(labels, bins = 255)
plt.show()

In [ ]:
# Create binary labels
# TRUE if it's a vessel, otherwise FALSE
binary_labels = 1 * (labels < 128) + 0 * (labels >= 128)
print(binary_labels.shape)
print(binary_labels[:8])
plt.hist(binary_labels, bins = 2)
plt.show()

In [ ]:
#n_neg = int(sum(binary_labels[:, 1]))
n_pos = int(sum(binary_labels))
n_neg = count - n_pos
print('Number of negative samples: ', n_neg)
print('Number of positive samples: ', n_pos)
print('Fraction of positive samples: ', n_pos / count * 100, '%')
new_ratio = 9
n_neg_new = int(new_ratio * n_pos)
print('Need to randomly choose ', n_neg_new, ' negative samples to make it', new_ratio, ': 1')

## Reduce the number of negative samples

In [ ]:
#neg_indeces = (np.where(binary_labels[:, 1] == 1))[0]
neg_indeces = (np.where(binary_labels == 0))[0]
np.random.shuffle(neg_indeces)
neg_indeces_new = neg_indeces[:n_neg_new]

## Create stratified training and test sets

In [ ]:
train_ratio = 0.8
n_samples = n_pos + n_neg_new

print(n_samples, n_pos, n_neg_new)
print('train: ', round(train_ratio * n_samples), ', test: ', round((1 - train_ratio) * n_samples))
print('Positive. y_train: ', round(train_ratio * n_pos), ', y_test: ', round((1 - train_ratio) * n_pos))
print('Negative. y_train: ', round(train_ratio * n_neg_new), ', y_test: ', round((1 - train_ratio) * n_neg_new))

In [ ]:
#pos_indeces = (np.where(binary_labels[:, 0] == 1))[0]
pos_indeces = (np.where(binary_labels == 1))[0]
np.random.shuffle(pos_indeces)
pos_indeces_train = pos_indeces[:round(train_ratio * n_pos)]
pos_indeces_test = pos_indeces[round(train_ratio * n_pos):]

neg_indeces_train = neg_indeces_new[:round(train_ratio * n_neg_new)]
neg_indeces_test = neg_indeces_new[round(train_ratio * n_neg_new):]

In [ ]:
train_indeces = np.append(pos_indeces_train, neg_indeces_train, axis=0)
np.random.shuffle(train_indeces)
X_train = originals[train_indeces, :, :, :]
#y_train = labels[train_indeces]
y_train = binary_labels[train_indeces]

test_indeces = np.append(pos_indeces_test, neg_indeces_test, axis = 0)
np.random.shuffle(test_indeces)
X_test = originals[test_indeces, :, :, :]
#y_test = labels[test_indeces]
y_test = binary_labels[test_indeces]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
X_train.dtype

In [ ]:
X_train_small = X_train[:100, :, :, :]
y_train_small = y_train[:100, :]
print(X_train_small.shape, y_train_small.shape)
print(y_train_small.sum())

## CNN

In [7]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # X is a 4-D tensor: [batch_size, width, height, channels]
    # Angio images are 23x23 pixels, and have three color channel
    #input_layer = features["x"]
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    print('input_layer: ', input_layer.shape)
    
    # Convolutional Layer #1
    # Computes 64 features using a 3x3 filter, no activation function
    # Padding is added to preserve width and height
    # Input Tensor Shape: [batch_size, 23, 23, 3]
    # Output Tensor Shape: [batch_size, 23, 23, 64]
    conv1 = tf.layers.conv2d(
    inputs = input_layer,
    filters = 64,
    kernel_size = [3, 3],
    padding = "same")  
    print('conv1: ', conv1.shape)

    # Pooling Layer #1
    # Max pooling layer with a 3x3 filter and stride of 2
    # Input Tensor Shape: [batch_size, 23, 23, 64]
    # Output Tensor Shape: [batch_size, 11, 11, 64]
    pool1 = tf.layers.max_pooling2d(
    inputs = conv1,
    pool_size = [3, 3],
    strides = 2)
    print('pool1: ', pool1.shape)
    
    # Convolutional Layer #2
    # Computes 128 features using a 3x3 filter, no activation function
    # Padding is added to preserve width and height
    # Input Tensor Shape: [batch_size, 11, 11, 64]
    # Output Tensor Shape: [batch_size, 11, 11, 128]
    conv2 = tf.layers.conv2d(
    inputs = pool1,
    filters = 128,
    kernel_size = [3, 3],
    padding = "same")
    print('conv2: ', conv2.shape)
    
    # Pooling Layer #2
    # Max pooling layer with a 3x3 filter and stride of 2
    # Padding is added to preserve width and height
    # Input Tensor Shape: [batch_size, 11, 11, 128]
    # Output Tensor Shape: [batch_size, 5, 5, 128]
    pool2  = tf.layers.max_pooling2d(
    inputs = conv2,
    pool_size = [3, 3],
    strides = 2)
    print('pool2: ', pool2.shape)
    
    # Convolutional Layer #3
    # Computes 256 features using a 3x3 filter and stride of 2, no activation function
    # Padding is added to preserve width and height
    # Input Tensor Shape: [batch_size, 5, 5, 128]
    # Output Tensor Shape: [batch_size, 3, 3, 256]
    conv3 = tf.layers.conv2d(
    inputs = pool2,
    filters = 256,
    kernel_size = [3, 3],
    strides = 1,
    padding = "valid")
    print('conv3: ', conv3.shape)
    
    # Pooling Layer #3
    # Max pooling layer with a 3x3 filter
    # No padding is added.
    # Input Tensor Shape: [batch_size, 3, 3, 256]
    # Output Tensor Shape: [batch_size, 1, 1, 256]
    pool3 = tf.layers.max_pooling2d(
    inputs = conv3,
    pool_size = [3, 3],
    strides = 1)
    print('pool3: ', pool3.shape)
    
    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 1, 1, 256]
    # Output Tensor Shape: [batch_size, 1 * 1 * 256]
    pool2flat = tf.reshape(pool3, [-1, 2 * 2 * 256])
    print('pool2flat: ', pool2flat.shape)

    # Dense Layer
    # Densely connected layer with 256 neurons
    # Input Tensor Shape: [batch_size, 1 * 1 * 256]
    # Output Tensor Shape: [batch_size, 256]    
    dense = tf.layers.dense(
    inputs = pool2flat,
    units = 256)
    print('dense: ', dense.shape)
    
    # Logits layer
    # Input Tensor Shape: [batch_size, 256]
    # Output Tensor Shape: [batch_size, 2]
    logits = tf.layers.dense(
    inputs = dense,
    units = 10)
    print('logits: ', logits.shape)
    
    classes = tf.argmax(input = logits, axis = 1, name = "class")
    print('classes: ', classes.shape)
    probabilities = tf.nn.softmax(logits, name = "softmax_tensor")
    print('probabilities: ', probabilities.shape)
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": classes,
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`
        "probabilities": probabilities
    }
    print('predictions: ', len(predictions))
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    #onehot_labels = tf.concat([1 - labels, labels], axis = 1)
    #print('labels: ', labels.shape)
    #print('onehot labels: ', onehot_labels.shape)
    #softmax_logits = tf.nn.softmax(logits)
    #loss = tf.losses.log_loss(labels = onehot_labels, predictions = softmax_logits)  
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    print('...Loss...')
    
    # Configure the Training Op (for TRAIN mode)
    if mode ==tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op)
    
    print('labels: ', labels.shape)
    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels = labels, predictions = predictions["classes"]),
        "precision": tf.metrics.precision(
            labels = labels, predictions = predictions["classes"]),
        "recall": tf.metrics.recall(
            labels = labels, predictions = predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode = mode, loss = loss, eval_metric_ops = eval_metric_ops)

## Train the model

In [2]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [8]:
# Create the estimator
angio_classifier = tf.estimator.Estimator(
    model_fn = cnn_model_fn, model_dir = "/tmp/mnist_custom_model")

# Set up logging for predictions
tensors_to_log = {"classes" : "class", 
                  "probabilities" : "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors = tensors_to_log, every_n_iter = 500)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    #x = {"x": X_train},
    #y = y_train,
    x={"x": train_data},
    y=train_labels,
    batch_size = 100,
    num_epochs = None,
    shuffle = False)
angio_classifier.train(
    input_fn = train_input_fn,
    steps = 10000,
    hooks = [logging_hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_custom_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a26cd0f98>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
input_layer:  (100, 28, 28, 1)
conv1:  (100, 28, 28, 64)
pool1:  (100, 13, 13, 64)
conv2:  (100, 13, 13, 128)
pool2:  (100, 6, 6, 128)
conv3:  (100, 4, 4, 256)
pool3:  (100, 2, 2, 256)
pool2flat:  (100, 1024)
dense:  (100, 256)
logits:  (100, 10)
classes:  (100,)
probabilities:  (100, 10)
predictions:  2
...Loss...
INFO:tensor

INFO:tensorflow:loss = 2.3158119, step = 1
INFO:tensorflow:global_step/sec: 3.74496
INFO:tensorflow:loss = 2.2943716, step = 101 (26.695 sec)
INFO:tensorflow:global_step/sec: 3.93306
INFO:tensorflow:loss = 2.2696714, step = 201 (25.425 sec)
INFO:tensorflow:global_step/sec: 3.92607
INFO:tensorflow:loss = 2.2628312, step = 301 (25.471 sec)
INFO:tensorflow:global_step/sec: 3.9348
INFO:tensorflow:loss = 2.2551622, step = 401 (25.414 sec)
INFO:tensorflow:global_step/sec: 3.92585
INFO:tensorflow:classes = [1 6 8 3 9 8 0 8 3 8 2 8 6 8 1 2 0 8 6 6 8 2 3 0 9 8 8 0 3 0 0 1 0 3 0 3 1
 0 3 6 3 2 3 8 7 0 1 1 0 0 3 3 7 0 3 2 0 0 8 7 3 8 3 9 0 0 6 1 8 3 3 3 3 0
 9 8 3 6 3 2 1 8 8 1 0 2 7 8 7 2 2 3 9 3 7 0 0 8 7 6], probabilities = [[0.09870234 0.11620529 0.09850287 0.09623595 0.10492372 0.08643953
  0.09926071 0.1023028  0.0995941  0.09783272]
 [0.10037383 0.09425984 0.10402713 0.10465043 0.10553092 0.08982833
  0.10586353 0.09338359 0.1037555  0.09832688]
 [0.09880534 0.09523705 0.10083119 0.1006334

INFO:tensorflow:loss = 2.2278974, step = 501 (25.487 sec)
INFO:tensorflow:global_step/sec: 3.97872
INFO:tensorflow:loss = 2.214364, step = 601 (25.119 sec)
INFO:tensorflow:global_step/sec: 3.91784
INFO:tensorflow:loss = 2.2041378, step = 701 (25.524 sec)
INFO:tensorflow:global_step/sec: 3.90625
INFO:tensorflow:loss = 2.1845593, step = 801 (25.600 sec)
INFO:tensorflow:global_step/sec: 3.9062
INFO:tensorflow:loss = 2.1391892, step = 901 (25.600 sec)
INFO:tensorflow:global_step/sec: 3.64271
INFO:tensorflow:classes = [6 8 6 9 0 6 8 3 8 6 0 3 3 8 9 8 1 3 0 3 7 7 4 1 0 3 0 6 3 8 9 9 1 3 0 3 0
 7 7 6 3 9 0 3 6 3 3 7 8 6 4 1 0 9 8 1 1 0 6 0 8 0 1 1 7 3 7 3 1 6 0 6 0 6
 8 1 6 8 3 9 4 0 0 1 9 3 6 8 1 0 1 6 6 6 1 7 2 8 6 7], probabilities = [[0.09218152 0.09439952 0.10272209 0.11304501 0.08823112 0.09018815
  0.11316412 0.09990532 0.1081064  0.09805682]
 [0.09839071 0.08737101 0.10168444 0.11006588 0.08834748 0.08966926
  0.10527388 0.10511977 0.11183086 0.10224678]
 [0.11550111 0.08017343 0.1050

INFO:tensorflow:loss = 2.1131718, step = 1001 (27.468 sec)
INFO:tensorflow:global_step/sec: 0.397285
INFO:tensorflow:loss = 2.0520513, step = 1101 (251.693 sec)
INFO:tensorflow:global_step/sec: 3.91086
INFO:tensorflow:loss = 1.9911749, step = 1201 (25.569 sec)
INFO:tensorflow:global_step/sec: 3.67751
INFO:tensorflow:loss = 1.90899, step = 1301 (27.192 sec)
INFO:tensorflow:global_step/sec: 3.91644
INFO:tensorflow:loss = 1.9038718, step = 1401 (25.533 sec)
INFO:tensorflow:Saving checkpoints for 1449 into /tmp/mnist_custom_model/model.ckpt.
INFO:tensorflow:global_step/sec: 3.90436
INFO:tensorflow:classes = [3 1 1 5 7 2 1 9 0 4 6 6 3 6 7 3 7 7 0 1 7 0 2 6 9 4 9 8 2 1 6 3 1 7 9 0 6
 1 4 9 7 0 0 8 9 7 7 9 0 6 8 1 5 7 7 2 9 8 0 7 7 0 3 1 9 9 0 6 7 6 8 9 5 6
 7 1 1 2 1 7 6 7 6 6 9 5 9 1 3 3 0 6 1 3 0 4 3 7 8 4], probabilities = [[0.07255368 0.07064217 0.09688246 0.13652603 0.07342443 0.09590103
  0.11851797 0.09866533 0.12550114 0.11138579]
 [0.04361738 0.3826703  0.0642733  0.05466964 0.08372

INFO:tensorflow:loss = 1.7970381, step = 1501 (25.627 sec)
INFO:tensorflow:global_step/sec: 3.96542
INFO:tensorflow:loss = 1.6618745, step = 1601 (25.204 sec)
INFO:tensorflow:global_step/sec: 3.97983
INFO:tensorflow:loss = 1.5955211, step = 1701 (25.127 sec)
INFO:tensorflow:global_step/sec: 3.93276
INFO:tensorflow:loss = 1.4894607, step = 1801 (25.427 sec)
INFO:tensorflow:global_step/sec: 4.0365
INFO:tensorflow:loss = 1.4556198, step = 1901 (24.774 sec)
INFO:tensorflow:global_step/sec: 3.90498
INFO:tensorflow:classes = [7 8 0 0 1 3 9 1 1 6 0 8 9 4 3 1 3 7 2 6 3 2 9 6 1 4 9 2 9 7 9 8 5 1 0 3 6
 2 0 0 0 3 0 0 2 1 7 9 3 2 1 3 1 2 2 3 1 3 1 8 7 7 4 9 1 9 4 7 3 1 2 1 1 8
 0 6 6 7 3 2 7 4 0 8 2 6 1 1 0 8 2 0 8 6 4 2 2 1 1 0], probabilities = [[0.02921284 0.04038308 0.08526883 0.08827569 0.12304743 0.06598217
  0.0280487  0.2992823  0.12391556 0.11658336]
 [0.05416832 0.01673903 0.06449181 0.08340489 0.15269099 0.09854791
  0.05008427 0.1311264  0.19049096 0.15825546]
 [0.53526706 0.00322611 

...

INFO:tensorflow:loss = 0.13584694, step = 7501 (25.378 sec)
INFO:tensorflow:Saving checkpoints for 7600 into /tmp/mnist_custom_model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.146255
INFO:tensorflow:loss = 0.14463367, step = 7601 (683.725 sec)
INFO:tensorflow:global_step/sec: 3.24818
INFO:tensorflow:loss = 0.14908682, step = 7701 (30.785 sec)
INFO:tensorflow:global_step/sec: 3.99703
INFO:tensorflow:loss = 0.10496533, step = 7801 (25.018 sec)
INFO:tensorflow:global_step/sec: 3.52267
INFO:tensorflow:loss = 0.12567057, step = 7901 (28.388 sec)
INFO:tensorflow:global_step/sec: 3.84109
INFO:tensorflow:classes = [1 1 9 4 9 8 1 8 1 1 3 2 0 9 2 3 9 8 8 9 2 8 3 2 9 8 6 6 0 5 3 2 7 6 6 2 8
 6 8 0 5 5 8 3 8 3 5 6 1 3 0 3 1 7 9 6 0 5 3 9 0 8 5 0 6 9 3 0 9 1 9 2 8 3
 0 4 2 5 4 6 9 7 5 8 9 9 3 0 8 1 2 2 2 3 1 4 8 5 4 6], probabilities = [[0.00003156 0.9980572  0.00000948 0.00001944 0.00147133 0.00000519
  0.00003055 0.00010608 0.00023668 0.00003241]
 [0.00017329 0.99815196 0.00004073 0.00000163 

INFO:tensorflow:loss = 0.1355959, step = 8001 (26.048 sec)
INFO:tensorflow:global_step/sec: 3.42116
INFO:tensorflow:loss = 0.23779696, step = 8101 (29.218 sec)
INFO:tensorflow:global_step/sec: 3.83082
INFO:tensorflow:loss = 0.09455998, step = 8201 (26.102 sec)
INFO:tensorflow:global_step/sec: 3.57517
INFO:tensorflow:loss = 0.20147072, step = 8301 (27.971 sec)
INFO:tensorflow:global_step/sec: 3.57766
INFO:tensorflow:loss = 0.20659553, step = 8401 (27.951 sec)
INFO:tensorflow:global_step/sec: 3.6242
INFO:tensorflow:classes = [3 7 3 9 6 4 4 1 7 0 9 3 5 8 2 7 4 7 7 8 5 7 9 6 6 6 8 2 5 8 5 1 2 6 0 1 6
 0 8 4 3 3 5 1 0 6 5 1 3 9 8 0 4 1 2 2 2 3 0 4 7 5 7 6 2 7 4 8 1 9 3 0 3 1
 6 2 2 3 2 4 0 5 5 6 3 8 5 0 0 1 3 2 8 3 2 8 9 0 8 9], probabilities = [[0.00001679 0.00001544 0.00072321 0.99815947 0.00000462 0.0004061
  0.00000053 0.00021157 0.00013297 0.00032927]
 [0.00000795 0.00000297 0.00047281 0.00021463 0.00005299 0.00000994
  0.         0.99858284 0.00000994 0.00064594]
 [0.0000141  0.000000

INFO:tensorflow:loss = 0.1453198, step = 8501 (27.608 sec)
INFO:tensorflow:global_step/sec: 3.37946
INFO:tensorflow:loss = 0.1137252, step = 8601 (29.577 sec)
INFO:tensorflow:global_step/sec: 3.58196
INFO:tensorflow:loss = 0.12098883, step = 8701 (27.916 sec)
INFO:tensorflow:global_step/sec: 3.76606
INFO:tensorflow:loss = 0.12984, step = 8801 (26.553 sec)
INFO:tensorflow:global_step/sec: 3.51594
INFO:tensorflow:loss = 0.08947193, step = 8901 (28.442 sec)
INFO:tensorflow:global_step/sec: 3.56727
INFO:tensorflow:classes = [3 4 5 1 2 0 4 3 8 7 6 7 8 7 4 8 8 2 8 9 2 7 0 5 0 5 7 1 3 2 6 6 3 3 1 6 1
 6 2 0 3 2 1 8 9 1 9 6 4 1 4 0 1 4 1 3 4 1 2 6 7 1 7 9 1 0 5 1 6 2 7 3 8 4
 8 5 6 6 3 7 5 8 0 9 5 0 4 1 2 2 6 3 8 4 3 7 3 8 6 9], probabilities = [[0.00003145 0.00017618 0.01409355 0.8226807  0.00001977 0.01393889
  0.00000403 0.09926017 0.01729957 0.0324956 ]
 [0.00032465 0.00038334 0.00160161 0.00000578 0.9954313  0.00015153
  0.00122378 0.00006165 0.00007101 0.00074523]
 [0.00000775 0.00000001

INFO:tensorflow:loss = 0.10747744, step = 9001 (28.046 sec)
INFO:tensorflow:global_step/sec: 3.16813
INFO:tensorflow:loss = 0.1189734, step = 9101 (31.552 sec)
INFO:tensorflow:global_step/sec: 3.57327
INFO:tensorflow:loss = 0.20446928, step = 9201 (27.984 sec)
INFO:tensorflow:global_step/sec: 3.67835
INFO:tensorflow:loss = 0.08065535, step = 9301 (27.187 sec)
INFO:tensorflow:global_step/sec: 3.47853
INFO:tensorflow:loss = 0.18088417, step = 9401 (28.747 sec)
INFO:tensorflow:global_step/sec: 3.58278
INFO:tensorflow:classes = [5 9 7 3 7 6 5 2 4 3 0 4 0 6 8 8 2 5 3 7 8 2 2 3 8 8 9 1 2 2 9 9 9 5 5 8 1
 8 5 7 4 1 2 7 5 1 1 1 7 0 6 3 1 4 9 2 4 6 1 4 0 7 8 4 1 2 1 7 8 4 1 2 7 9
 4 9 6 2 2 8 3 6 4 1 9 0 9 6 8 8 7 5 5 5 1 0 3 5 4 3], probabilities = [[0.00035057 0.         0.00003849 0.0000847  0.00000057 0.998497
  0.00018338 0.00000041 0.0007594  0.00008552]
 [0.00000467 0.00000037 0.00009469 0.00005437 0.00546565 0.00012794
  0.0000069  0.00179815 0.00019334 0.99225396]
 [0.0002875  0.001601

INFO:tensorflow:loss = 0.18989426, step = 9501 (27.924 sec)
INFO:tensorflow:global_step/sec: 3.49789
INFO:tensorflow:loss = 0.12941812, step = 9601 (28.575 sec)
INFO:tensorflow:global_step/sec: 3.55651
INFO:tensorflow:loss = 0.09736651, step = 9701 (28.118 sec)
INFO:tensorflow:Saving checkpoints for 9740 into /tmp/mnist_custom_model/model.ckpt.
INFO:tensorflow:global_step/sec: 3.829
INFO:tensorflow:loss = 0.10398066, step = 9801 (26.117 sec)
INFO:tensorflow:global_step/sec: 3.6331
INFO:tensorflow:loss = 0.11607797, step = 9901 (27.524 sec)
INFO:tensorflow:Saving checkpoints for 10000 into /tmp/mnist_custom_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.10789665.


In [9]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    #x = {"x": X_test}, 
    #y = y_test,
    x={"x": eval_data},
    y=eval_labels,
    num_epochs = 1,
    shuffle = False)
eval_results = angio_classifier.evaluate(input_fn = eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
input_layer:  (?, 28, 28, 1)
conv1:  (?, 28, 28, 64)
pool1:  (?, 13, 13, 64)
conv2:  (?, 13, 13, 128)
pool2:  (?, 6, 6, 128)
conv3:  (?, 4, 4, 256)
pool3:  (?, 2, 2, 256)
pool2flat:  (?, 1024)
dense:  (?, 256)
logits:  (?, 10)
classes:  (?,)
probabilities:  (?, 10)
predictions:  2
...Loss...
labels:  (?,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-11-04:31:45
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_custom_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-11-04:31:51
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.9601, global_step = 10000, loss = 0.13274486, precision = 0.99799955, recall = 0.9955654
{'accuracy': 0.9601, 'loss': 0.13274486, 'precision': 0.99799955, 'recall': 0.9955654, 'global_step': 10000}
